In [20]:
import ee
import os
ee.Initialize()

### Make a folder to hold fuel layers generated from your treatment scenario ensemble

In [21]:
scenarios_path = "projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/treatment_scenarios"
scenarios = ee.ImageCollection(scenarios_path)
#create a folder representing scenarios imgColl name scheme
folder=scenarios_path.replace('treatment_scenarios','fuelscapes_scenarios')
os.popen(f"earthengine create folder {folder}").read()
print(f'Created Folder: {folder}')

Created Folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios


### For each treated landscape img in your treatment scenarios imageCollection, create a new folder to hold the fuels layers

In [22]:
scenarios_img_paths = os.popen(f"earthengine ls {scenarios_path}").read().split('\n')[0:-1]
scn_sub_folders = [(folder+'/'+os.path.basename(path)) for path in scenarios_img_paths]
# print(scenarios_img_paths[3])
# print(scn_sub_folders[3])

for sub_folder_name in scn_sub_folders:
    os.popen(f"earthengine create folder {sub_folder_name}").read()
    print(f'Created scenario fuelscapes sub-folder: {sub_folder_name}')
    

Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario1
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario10
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario11
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario12
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario2
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario3
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatm

### Loop thru each fuel updater script the way we do for FireFactor: create_canopy_guide, calc_CC_CH, calc_CBD_CBH, calc_FM40 (don't need to run fm40 wui update)

In [23]:
# get config.yml file path, needed for every script
config_path = f"{os.getcwd()}\config.yml" # on windows its a forward slash

### You can run Canopy Guide and FM40 updater scripts at same time since they do not depend on one another

In [24]:
# Canopy Guide
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python create_canopy_guide.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    #break

# FM40
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python calc_FM40.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    #break


python create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/treatment_scenarios/scenario1 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario1
python create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/treatment_scenarios/scenario10 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario10
python create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/treatment_scenarios/scenario11 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220808/fuelscapes_scenarios/scenario11
python create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_sce

### You must wait until all Canopy Guide exports have successfully completed before running the CC_CH script

In [14]:
# CC and CH
# for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
#     cmd = f"python calc_CC_CH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
#     print(cmd)
#     os.popen(cmd).read()
#     print('\n')
#     break

### wait until all CC and CH exports have completed before running CBD_CBH 

In [15]:
# CBD and CBH
# for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
#     cmd = f"python calc_CBD_CBH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
#     print(cmd)
#     os.popen(cmd).read()
#     print('\n')
#     break

### After all fuel layers in each scenario's fuelscape has successfully exported, collate each 5-layer fuelscape into a multiband img for Google Drive Export

In [16]:
# must mosaic the canopy guide and fm40 collections (in this case just .first()) 
# since they are exported as zone-wise img collections by default